In [ ]:
import coolingFunction

In [ ]:
import copy
import numpy as np
import h5py
import scipy
import scipy.special
import sys
import verdict
import os
import tqdm
import unyt

In [ ]:
import kalepy as kale

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.patheffects as path_effects
import matplotlib.cm as cm
import matplotlib.colors as plt_colors
import matplotlib.gridspec as gridspec
import matplotlib.transforms
import palettable

In [ ]:
import linefinder.analyze_data.worldlines as a_worldlines
import linefinder.analyze_data.worldline_set as worldline_set
import linefinder.analyze_data.plot_worldlines as p_worldlines
import linefinder.utils.presentation_constants as p_constants

In [ ]:
import galaxy_dive.analyze_data.ahf as analyze_ahf
import galaxy_dive.plot_data.ahf as plot_ahf
import galaxy_dive.analyze_data.particle_data as particle_data
import galaxy_dive.plot_data.generic_plotter as generic_plotter
import galaxy_dive.plot_data.plotting as plotting
import galaxy_dive.utils.data_operations as data_operations
import galaxy_dive.utils.executable_helpers as exec_helpers

In [ ]:
import linefinder.utils.file_management as file_management
import linefinder.config as config

In [ ]:
import trove

In [ ]:
from py2tex import py2tex

In [ ]:
import helpers

In [ ]:
%matplotlib inline

# Load Data

In [ ]:
pm = dict(
    snum = 600,
    tables_dir = '/work/03057/zhafen/CoolingTables/',
    study_duplicates = False,
    ahf_index = 600,
    
    # If we want to ensure some minimum number of snapshots in the galaxy after accreting
    # (remember to account for the last 10 snapshots with small dt)
    minInd = 0,
    maxInd = 54, # Corresponds to 1 Gyr. We don't look at accretion prior.
    
    # For the fancy scatter plot we're visualizing.
    variable_alpha = True,
)

In [ ]:
pm = trove.link_params_to_config(
    '/home1/03057/zhafen/papers/Hot-Accretion-in-FIRE/analysis/hot_accretion.trove',
    script_id = 'nb.11',
    **pm
)

In [ ]:
# Used so often it's nice to not enclose
snum = pm['snum']
ind = pm['ahf_index'] - snum

In [ ]:
w = a_worldlines.Worldlines(
    tag = pm['tag'],
    data_dir = pm['base_data_dir'],
    halo_data_dir = pm['halo_data_dir'],
    ahf_index = pm['ahf_index'],
)

In [ ]:
w.retrieve_halo_data()

In [ ]:
m_plot_label  = r'$M_{\rm h} = 10^{' + '{:.02g}'.format( np.log10( w.m_vir[snum] ) )
m_plot_label += '} M_\odot$'
plot_label = m_plot_label + ', z={:.02}'.format( w.redshift[snum] )
print( plot_label )

In [ ]:
base_processed_data_dir = pm['config_parser'].get( 'DEFAULT', 'processed_data_dir' )
default_data_fp = os.path.join( base_processed_data_dir, 'summary.hdf5' )
default_data = verdict.Dict.from_hdf5( default_data_fp, create_nonexistent=True )

In [ ]:
data_fp = os.path.join( pm['processed_data_dir'], 'summary.hdf5' )
data = verdict.Dict.from_hdf5( data_fp, create_nonexistent=True )

## Labels

In [ ]:
tchange_key = pm['central_indices'].split( '_' )[0]
t_tchange_key = 't_' + tchange_key

In [ ]:
t_tchange_label = helpers.get_t_tchange_label( pm )

## Calculate Central Indices

In [ ]:
if pm['central_indices'] == 'tcools_inds':
    inds = w.calc_tcools_inds(
        lookback_time_max = pm['lookback_time_max'],
        choose_first = pm['choose_first'],
        B = pm['logTcools'],
    )
else:
    calc_fn = getattr( w, 'calc_{}'.format( pm['central_indices'] ) )
    inds = calc_fn(
        lookback_time_max = pm['lookback_time_max'],
        choose_first = pm['choose_first'],
    )

In [ ]:
valid = inds != config.INT_FILL_VALUE
valid_inds = inds[valid]
particle_inds = np.arange( w.n_particles )[valid]

## Calculate $\vec j$

In [ ]:
specific_mom = w.get_data( 'J' )
w.data['Jmag'] = w.get_data( 'Jmag' )

In [ ]:
tot_momentum_fp = os.path.join( base_processed_data_dir, 'tot_momentums.hdf5' )
tot_ang_momentum = verdict.Dict.from_hdf5( tot_momentum_fp )[pm['variation']]['snum{:03d}'.format( snum )]

In [ ]:
tot_ang_momentum_normed = tot_ang_momentum / np.linalg.norm( tot_ang_momentum )
w.data['Jz'] = np.array([ np.dot( specific_mom[:,:,i].transpose(), tot_ang_momentum_normed ) for i in range( w.snums.size ) ]).transpose()

In [ ]:
w.data['Jz/J'] = w.data['Jz'] / w.data['Jmag']

# Plot

## Settings

In [ ]:
def get_cmap_and_norm( data_key ):

    if data_key == 'T':
        cmap = matplotlib.colors.LinearSegmentedColormap.from_list(
            'berlin_white',
            [
                palettable.scientific.diverging.Berlin_3.mpl_colors[0],
                'w',
                palettable.scientific.diverging.Berlin_3.mpl_colors[-1],
            ],
        )
        norm = plt_colors.LogNorm( vmin=1e4, vmax=1e6 )
    elif data_key == 'Jz/J':
        cmap = matplotlib.colors.LinearSegmentedColormap.from_list(
            'tofino_white',
            [
                palettable.cartocolors.diverging.Tropic_7.mpl_colors[-2],
                'w',
                palettable.cartocolors.diverging.Tropic_7.mpl_colors[1],
            ],
        )
        norm = plt_colors.Normalize( vmin=-1, vmax=1 )

    return cmap, norm

In [ ]:
cmap, norm = get_cmap_and_norm( 'T' )

In [ ]:
# Setup time steps
delta_t = 0.01
t_movie_after = np.arange( 0., 0.5 + delta_t, delta_t )
t_movie_before = -1. * np.arange( 0., 1.5 + delta_t, delta_t )[1:][::-1]
t_movie = np.concatenate( [ t_movie_before, t_movie_after ]  )

## Get Time Offset
Find offset for each worldline (to account for t_tchange occuring between snapshots)

In [ ]:
# Mask Data
w.data_masker.clear_masks()
w.data_masker.mask_data( 'n_out', -1, 1 ) # Only include particles that have never left the main galaxy
w.data_masker.mask_data( 'PType', data_value=0 )

In [ ]:
# Find particles that are valid near tchange
mask_overall = w.data_masker.get_mask()
mask_at_tchange = mask_overall[particle_inds,valid_inds]
mask_after_tchange = mask_overall[particle_inds,valid_inds-1]
masked_near_tchange = np.logical_and( mask_at_tchange, mask_after_tchange )
valid_near_tchange = np.invert( masked_near_tchange )

In [ ]:
# Find the temperatures to interpolate to get the offset
T_at_tchange = w.get_data( 'T' )[particle_inds,valid_inds][valid_near_tchange]
T_after_tchange = w.get_data( 'T' )[particle_inds,valid_inds-1][valid_near_tchange]
logT_interp = np.log10( np.array([ T_at_tchange, T_after_tchange ]).transpose() )

In [ ]:
# Find the times to interpolate to get the offset
t_tchange = w.get_data( t_tchange_key )
t_tchange_at_tchange = t_tchange[particle_inds,valid_inds][valid_near_tchange]
t_tchange_after_tchange = t_tchange[particle_inds,valid_inds-1][valid_near_tchange]
t_tchange_interp = np.array([ t_tchange_at_tchange, t_tchange_after_tchange ]).transpose()
t_tchange = w.get_data( t_tchange_key )[valid_inds]

In [ ]:
# Get the offset
time_offsets = []
n_wrong = 0
for i, tchange_interp_i in enumerate( tqdm.tqdm( t_tchange_interp ) ):
    interp_fn = scipy.interpolate.interp1d( logT_interp[i], t_tchange_interp[i],  )
    try:
        time_offsets.append( interp_fn( pm['logTcools'] ) )
    except ValueError:
        time_offsets.append( np.nan )
        n_wrong += 1

time_offsets = np.array( time_offsets )
print( n_wrong, n_wrong / t_tchange_interp.shape[0] )

time_offsets_all = np.full( particle_inds.size, np.nan )
time_offsets_all[valid_near_tchange] = time_offsets

t_tchange_corrected = t_tchange - time_offsets_all[:,np.newaxis]

# The below code shows individual cases where tchange appears to be calculated wrong
if np.isnan( time_offsets ).sum() > 0:
    i = np.argmax( time_offsets )
    weird_original_ind = particle_inds[valid_near_tchange][i]
    print( np.log10( w.get_data( 'T' )[weird_original_ind] )[inds[weird_original_ind]-2:inds[weird_original_ind]+1] )

## Get Data

In [ ]:
w.data_masker.clear_masks()
w.data_masker.mask_data( 'n_out', -1, 1 ) # Only include particles that have never left the main galaxy
w.data_masker.mask_data( 'PType', data_value=0 )

In [ ]:
ds = []
for i, t_frame in enumerate( tqdm.tqdm( t_movie ) ):

    t_frame_inds = np.argmin( np.abs( t_tchange_corrected - t_frame ), axis=1 )

    # Prepare to mask
    if len( w.data_masker.masks ) == 3:
        del w.data_masker.masks[2]

    # Mask data that's out of bounds
    out_of_bounds = t_frame_inds == 0
    out_of_bounds_full = np.zeros( w.n_particles ).astype( 'bool' )
    out_of_bounds_full[particle_inds] = out_of_bounds
    out_of_bounds_mask = np.tile( out_of_bounds_full, ( w.n_snaps, 1 ) ).transpose()
    w.data_masker.mask_data( 'out_of_bounds', custom_mask=out_of_bounds_mask,  )

    # Base parameters
    d = {}
    for key in [ 'Rx', 'Ry', 'Rz', 'M', 'Den', 'T', 'Jz/J' ]:
        d[key] = w.get_selected_data( key, compress=False )[particle_inds,t_frame_inds].compressed()

    # Volume and smoothing length
    den_msunkpc3 = ( d['Den']*unyt.mp/unyt.cm**3  ).to( 'Msun/kpc**3' )
    d['Vol'] = d['M'] * unyt.Msun / den_msunkpc3
    d['h'] = ( 3. * d['Vol'] / 4. / np.pi )**(1./3.)

    ds.append( d )

## Make Frames

In [ ]:
# Frame and scalebar size objects
def round_down(num, divisor):
    return num - (num%divisor)

In [ ]:
# The t_movie cut is set to be useless by default.
lim = np.max( [ np.nanpercentile( np.abs( ds[i]['Rx'] ), 95 ) for i in np.arange(len(ds))[t_movie >= -0.5] ] )

In [ ]:
for data_key in [ 'Jz/J', 'T' ]:

    cmap, norm = get_cmap_and_norm( data_key )

    i_focused = 0

    for i, d in enumerate( tqdm.tqdm( ds ) ):
        
        # DEBUG
        if i not in [ 100, 150, 200 ]:
            continue

        fig = plt.figure( figsize=(12,10), facecolor='k' )
        ax = plt.gca()

        # Point size
        width_in_data = 2 * lim
        width_in_pixels = ax.get_window_extent().width
        pixels_to_points = fig.dpi / 72.
        scale = 10.
        radius = d['h'] * ( width_in_pixels / width_in_data ) * pixels_to_points * scale
        s = ( radius )**2.

        # Colors
        colors = cmap( norm( d[data_key] ) )

        # Alpha
        if pm['variable_alpha']:
            column_den = d['M'] / d['h']**2.
            alpha = plt_colors.LogNorm( vmin=np.nanmin( column_den ), vmax=np.nanmax( column_den ) )( column_den ) * 0.065 * ( 50000 / w.n_particles)
            alpha[alpha>1.] = 1.
            alpha[alpha<0.] = 0.
            colors[:,3] = alpha
        else:
            colors[:,3] = 0.01

        # Plot itself
        ax.scatter(
            d['Rx'],
            d['Rz'],
            s = s,
            c = colors,
            edgecolors = 'none',
        )

        # Scale bar
        size = round_down( min( 30., 0.95 * lim ), 10 )
        if np.isclose( size, 0. ):
            size = 10.
        xy = ( -size, -0.95*lim )
        line = ax.plot(
            xy[0] + np.array([ 0., size ]),
            [ xy[1], xy[1] ],
            linewidth = 10,
            color = 'w',
            path_effects = [
                path_effects.Stroke(linewidth=12, foreground='black'),
                path_effects.Normal()
            ]
        )
        text = ax.annotate(
            text = '{:.2g} kpc'.format( size ),
            xy = xy,
            xycoords = 'data',
            xytext = ( 5, 10 ),
            textcoords = 'offset points',
            va = 'bottom',
            ha = 'left',
            color = 'w',
            fontsize = 42,
        )
        text.set_path_effects([
            path_effects.Stroke(linewidth=2.5, foreground='black'),
            path_effects.Normal()
        ])

        # Plot label
        annotate_label = (
            r'$t - ' +
            helpers.get_tchange_label( pm )[1:] +
            '= {} Myrs'.format( int( t_movie[i]*1e3 ) )
        )
        text = ax.annotate(
            text = annotate_label,
            xy = ( 1, 1 ),
            xycoords = 'axes fraction',
            xytext = ( -10, -10 ),
            textcoords = 'offset points',
            va = 'top',
            ha = 'right',
            color = 'w',
            fontsize = 42,
        )
        text.set_path_effects([
            path_effects.Stroke(linewidth=2.5, foreground='black'),
            path_effects.Normal()
        ])
        
        # Temperature
        temp_label = r'$\langle T \rangle =' + py2tex.to_tex_scientific_notation( np.nanmedian( d['T'] ), sig_figs=2 ) + r'$ K'
        text = ax.annotate(
            text = temp_label,
            xy = ( 1, 0 ),
            xycoords = 'axes fraction',
            xytext = ( -10, 10 ),
            textcoords = 'offset points',
            va = 'bottom',
            ha = 'right',
            color = 'w',
            fontsize = 34,
        )
        text.set_path_effects([
            path_effects.Stroke(linewidth=2, foreground='black'),
            path_effects.Normal()
        ])

        # Limits
        ax.set_xlim( -lim, lim )
        ax.set_ylim( -lim, lim )
        ax.set_aspect( 'equal' )

        # Ticks
        plt.tick_params(
            which = 'both',
            left = False,
            labelleft = False,
            bottom = False,
            labelbottom = False,
        )

        # Change colors
        ax.set_facecolor( 'k' )
        plt.setp( ax.spines.values(), color='w' )
        [m.set_linewidth(3) for m in ax.spines.values()]

        save_file_tag = {
            'T': '',
            'Jz/J': '_alignment',
        }[data_key]
        plotting.save_fig(
            out_dir = os.path.join( pm['data_dir'], 'projected_frames' ),
            save_file = '{}{}_{:0>3d}.png'.format( pm['variation'], save_file_tag, i ),
            fig = fig,
            resolution = 150.,
        )

        # Save movie focused on time of cooling or accreting
        if np.abs( t_movie[i] ) <= 0.150:
            plotting.save_fig(
                out_dir = os.path.join( pm['data_dir'], 'projected_frames' ),
                save_file = '{}{}_focused_{:0>3d}.png'.format( pm['variation'], save_file_tag, i_focused ),
                fig = fig,
                resolution = 150.,
            )
            i_focused += 1

#         plt.close()